In [ ]:
import pandas as pd
from procs.DictsAndLists import lista, listaTCA

In [ ]:
df = pd.read_csv(
    "B00.txt", 
    # dtype=pd.StringDtype(), 
    sep="\t", 
    names=lista[0]["dtype"].keys()
)

In [ ]:
df.info(verbose=True)

In [20]:
import pandas as pd
from procs.DictsAndLists import lista, listaTCA

df_arrow = pd.read_csv(
    "b00.txt",
    dtype=lista[0]["dtype"],
    sep="\t",
    header=0,
    index_col=0,
    names=lista[0]["dtype"].keys(),
    engine="pyarrow",
    dtype_backend="pyarrow",
)

KeyError: "Only a column name can be used for the key in a dtype mappings argument. 'id_persona ' not found in columns."

In [ ]:
df_arrow

In [15]:
df_arrow.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968827 entries, 0 to 968826
Data columns (total 61 columns):
 #   Column                        Non-Null Count   Dtype          
---  ------                        --------------   -----          
 0   id_persona                    968827 non-null  int64[pyarrow] 
 1    QID                          968827 non-null  int64[pyarrow] 
 2    BASE                         968827 non-null  int64[pyarrow] 
 3    CORTE                        968827 non-null  int64[pyarrow] 
 4    CUIT                         968827 non-null  int64[pyarrow] 
 5    TDOC                         968827 non-null  int64[pyarrow] 
 6    NDOC                         968827 non-null  int64[pyarrow] 
 7    SEXO                         0 non-null       null[pyarrow]  
 8    DENO                         968827 non-null  string[pyarrow]
 9    FNAC                         968827 non-null  string[pyarrow]
 10   MONTO                        968827 non-null  string[pyarrow]
 11  

In [24]:
import sys
from rich import print
from rich.panel import Panel
from rich.text import Text
from rich.prompt import Prompt
import pandas as pd
import glob
import os
import shutil
import tarfile
import csv
from procs.DictsAndLists import lista, listaTCA
from pandasql import sqldf
import timeit

from rich.console import Console

# Inicializamos la consola de rich
console = Console(log_path=False)


def p30stat(paquete):
    # Abrimos el GZ en python para manosearlo por dentro.
    try:
        TarP3 = tarfile.open(paquete, "r:gz")
    except Exception as e:
        console.log("Hubo un error leyendo el archivo de vuelta.")
        Prompt.ask("\nPresione [red][ENTER][/red] para volver al menu.")
        return 0
    # console.log(f"Encontrado: {paquete}")
    p30 = {}
    # Buscamos los archivos dentro del GZ.
    # console.log("Leyendo vuelta SyNTIS.")
    # console.log(f"Leyendo DATOS:")
    # Empezamos con el dict "lista"
    for file in lista:
        try:
            filename = [name for name in TarP3.getnames() if file["Nombre"] in name]
            txt = TarP3.extractfile(filename[0])
            # print(txt.readline().decode().split('\t'))
            df = pd.read_csv(
                txt,
                sep="\t",
                encoding="ansi",
                dtype=file["dtype"],
                names=file["dtype"].keys(),
                # header=0,
                skiprows=1,
                # index_col=0,
                engine="pyarrow",
                dtype_backend="pyarrow",
            )
            df.set_index(df.columns[0], inplace=True)
            p30[file["Nombre"]] = df
            # print(f"[green]{file['Nombre']}", end=" | ", flush=True)
        except IndexError:
            # print(f"[red]{file['Nombre']}", end=" | ", flush=True)
            continue
    # Continuamos con el dict "listaTCA"
    print(f"\n")
    # console.log(f"Leyendo CODIGOS:")
    for file in listaTCA:
        try:
            filename = [name for name in TarP3.getnames() if file["Nombre"] in name]
            txt = TarP3.extractfile(filename[0])

            df = pd.read_csv(
                txt,
                sep="\t",
                encoding="ansi",
                # dtype=file["dtype"],
                index_col=0,
                engine="pyarrow",
                dtype_backend="pyarrow",
            )
            # print(f"[green]{file['Nombre']}", end=" | ", flush=True)
            df.insert(loc=0, column="tabla", value=file["Nombre"])
            p30[file["Nombre"]] = df
        except IndexError:
            # print(f"[red]{file['Nombre']}", end=" | ", flush=True)
            continue
    return p30
    # Cerramos el GZ para que no haya error al moverlo.
    TarP3.close()


paquete30 = p30stat("Vuelta_EfectoresPaquete_30.tar.gz")
# p30 = pd.read_fwf('EfectoresPaquete_30.txt',encoding='ANSI',widths=(15,9,50,50,8,1,2,10,20,40,6,5,5,5,5,20,20,30,8,20,20,30,30,50,50,15,50,50,30,50,50,11,5,70,6,2,12,12))
# cuits= pd.read_csv("cuits.txt",sep="\t")

#### GUARDAMOS LOS DICT EN VARIABLES USABLES 
Y en caso de no encontrar datos, crea dataframes vacios para evitar errores mas adelante.

In [28]:
var = paquete30.get("B00", pd.DataFrame(columns=lista[0]["dtype"].keys()))
dat = paquete30.get("DATOS", pd.DataFrame(columns=lista[1]["dtype"].keys()))
bar = paquete30.get("EMBARCACIONES", pd.DataFrame(columns=lista[2]["dtype"].keys()))
avi = paquete30.get("AERONAVES", pd.DataFrame(columns=lista[3]["dtype"].keys()))
dep = paquete30.get(
    "EMPLEO_DEPENDIENTE", pd.DataFrame(columns=lista[4]["dtype"].keys())
)
ind = paquete30.get(
    "EMPLEO_INDEPENDIENTE", pd.DataFrame(columns=lista[5]["dtype"].keys())
)
fal = paquete30.get("FALLECIDOS", pd.DataFrame(columns=lista[6]["dtype"].keys()))
inm = paquete30.get("INMUEBLES", pd.DataFrame(columns=lista[7]["dtype"].keys()))
jub = paquete30.get(
    "JUBILACIONES_PENSIONES", pd.DataFrame(columns=lista[8]["dtype"].keys())
)
aut = paquete30.get(
    "PADRON_AUTOMOTORES", pd.DataFrame(columns=lista[9]["dtype"].keys())
)
jur = paquete30.get(
    "PERSONAS_JURIDICAS", pd.DataFrame(columns=lista[10]["dtype"].keys())
)
pnc = paquete30.get("PNC", pd.DataFrame(columns=lista[11]["dtype"].keys()))
rub = paquete30.get("RUBPS", pd.DataFrame(columns=lista[12]["dtype"].keys()))
asg = paquete30.get("ASIGNACIONES", pd.DataFrame(columns=lista[13]["dtype"].keys()))

In [29]:
var.info(verbose=True)
dat.info(verbose=True)
bar.info(verbose=True)
avi.info(verbose=True)
dep.info(verbose=True)
ind.info(verbose=True)
fal.info(verbose=True)
inm.info(verbose=True)
jub.info(verbose=True)
aut.info(verbose=True)
jur.info(verbose=True)
pnc.info(verbose=True)
rub.info(verbose=True)
asg.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 968827 entries, 77753084 to 31571174
Data columns (total 60 columns):
 #   Column                        Non-Null Count   Dtype                                                       
---  ------                        --------------   -----                                                       
 0    QID                          968827 non-null  int64[pyarrow]                                              
 1    BASE                         968827 non-null  dictionary<values=int64, indices=int32, ordered=0>[pyarrow] 
 2    CORTE                        968827 non-null  dictionary<values=int64, indices=int32, ordered=0>[pyarrow] 
 3    CUIT                         968827 non-null  int64[pyarrow]                                              
 4    TDOC                         968827 non-null  dictionary<values=int64, indices=int32, ordered=0>[pyarrow] 
 5    NDOC                         968827 non-null  int64[pyarrow]                            

In [ ]:
activos = sqldf(
    """
                SELECT 
                [ PROV_ACTIVIDAD  ] as PROVINCIA,
                COUNT(DISTINCT d.[id_persona ])                                                  as EFECTORES,
                SUM(case when c.cuit is not null then 1 else 0 end)                              as EFECTORES_activos,
                COUNT(DISTINCT p.id_persona)                                                     as PNC, 
                SUM(case when c.cuit is not null and p.id_persona is not null then 1 else 0 end) as PNC_activos,
                COUNT(DISTINCT j.id_persona)                                                     as JUB_PEN,
                SUM(case when c.cuit is not null and j.id_persona is not null then 1 else 0 end) as JUB_PEN_activos,
                COUNT(DISTINCT e.id_persona)                                                     as EMPLEO_DEP,
                SUM(case when c.cuit is not null and e.id_persona is not null then 1 else 0 end) as EMPLEO_DEP_activos,
                COUNT(DISTINCT a.id_persona)                                                     as ASIG_FAM,
                SUM(case when c.cuit is not null and a.id_persona is not null then 1 else 0 end) as ASIG_FAM_activos,
                COUNT(DISTINCT f.id_persona)                                                     as FALLECIDOS,
                SUM(case when c.cuit is not null and f.id_persona is not null then 1 else 0 end) as FALLECIDOS_activos,
                COUNT(DISTINCT h.id_persona)                                                     as JURIDICAS,
                SUM(case when c.cuit is not null and h.id_persona is not null then 1 else 0 end) as JURIDICAS_activos,
                COUNT(DISTINCT r.id_persona)                                                     as RUBPS,
                SUM(case when c.cuit is not null and r.id_persona is not null then 1 else 0 end) as RUBPS_activos
                FROM      var d 
                LEFT JOIN cuits c on c.cuit       = d.[ CUIT  ]
                LEFT JOIN (select distinct id_persona from pnc) p on p.id_persona = d.[id_persona ]
                LEFT JOIN (select distinct id_persona from jub) j on j.id_persona = d.[id_persona ]
                LEFT JOIN (select distinct id_persona from dep) e on e.id_persona = d.[id_persona ]
                LEFT JOIN (select distinct id_persona from asg) a on a.id_persona = d.[id_persona ]
                LEFT JOIN (select distinct id_persona from fal) f on f.id_persona = d.[id_persona ]
                LEFT JOIN (select distinct id_persona from jur) h on h.id_persona = d.[id_persona ]
                LEFT JOIN (select distinct id_persona from rub) r on r.id_persona = d.[id_persona ]
                GROUP BY [ PROV_ACTIVIDAD  ]
              """
)
activos.set_index("PROVINCIA", inplace=True)
activos.loc["TOTAL"] = activos.sum(numeric_only=True)
activos

#### PREPARAMOS EL EXCEL

In [ ]:
# preparamos el excel
writer = pd.ExcelWriter("Resumen_P30.xlsx", engine="xlsxwriter")

#### PRIMERO SACAMOS TODOS LOS STATS GENERALES POR PROVINCIA

In [ ]:
stats_prov = sqldf(
    """
                SELECT 
                [ PROV_ACTIVIDAD  ] as PROVINCIA,
                [ LOCALIDAD  ] as LOCALIDAD,
                t.sexo,
                COUNT(DISTINCT d.[id_persona ]) as EFECTORES,
                COUNT(DISTINCT p.id_persona)    as PNC, 
                COUNT(DISTINCT j.id_persona)    as JUB_PEN,
                COUNT(DISTINCT e.id_persona)    as EMPLEO_DEP,
                COUNT(DISTINCT a.id_persona)    as ASIG_FAM,
                COUNT(DISTINCT f.id_persona)    as FALLECIDOS,
                COUNT(DISTINCT h.id_persona)    as JURIDICAS,
                COUNT(DISTINCT r.id_persona)    as RUBPS
                FROM      var d 
                LEFT JOIN pnc p on p.id_persona = d.[id_persona ]
                LEFT JOIN dat t on t.id_persona = d.[id_persona ]
                LEFT JOIN jub j on j.id_persona = d.[id_persona ]
                LEFT JOIN dep e on e.id_persona = d.[id_persona ]
                LEFT JOIN asg a on a.id_persona = d.[id_persona ]
                LEFT JOIN fal f on f.id_persona = d.[id_persona ]
                LEFT JOIN jur h on h.id_persona = d.[id_persona ]
                LEFT JOIN rub r on r.id_persona = d.[id_persona ]
                GROUP BY [ PROV_ACTIVIDAD  ],[ LOCALIDAD  ],t.sexo
              """
)
stats_prov.set_index("PROVINCIA", inplace=True)
stats_prov.loc["TOTAL"] = stats_prov.sum(numeric_only=True)
stats_prov.to_csv("Stats_por_provincia.txt", sep="\t")

#### EVALUACION MUEBLES

In [ ]:
muebles = sqldf(
    """
                select * from aut
                where ID_PERSONA IN ( 
                select dat.id_persona
                from dat inner join 
                aut on aut.ID_PERSONA=dat.id_persona
                GROUP BY dat.id_persona
                HAVING COUNT(DOMINIO)>2)
                ORDER BY ID_PERSONA"""
)
muebles.set_index("ID_PERSONA", inplace=True)

#### EVALUACION INMUEBLES

In [ ]:
inmuebles = sqldf(
    """
                select * from inm
                where ID_PERSONA IN ( 
                select id_persona
                from inm
                GROUP BY id_persona
                HAVING COUNT(ID_PERSONA)>1)
                ORDER BY ID_PERSONA"""
)
inmuebles.set_index("ID_PERSONA", inplace=True)

#### ARMAMOS EL DF DE LOS EVALUADOS

In [ ]:
eval = sqldf(
    """
            SELECT
            distinct(dat.id_persona),
            dat.cuit,
            dat.ndoc,
            dat.deno,
            case when dat.sexo=1 then 'M' else 'F' end as sexo,
            dat.fnac,
            dat.grado_confiab,
            case when inm.id_persona is not null then 'Si' else 'No' end as Inmuebles,
            case when aut.id_persona is not null then 'Si' else 'No' end as Muebles,
            case when dep.base=105 then 'Si' else 'No' end as Servicio_Domestico,
            case when dep.monto > 45000 then 'Si' else 'No' end as Dependiente,
            case when jur.id_persona is not null then 'Si' else 'No' end as Juridicas,
            case when jub.monto > 45000 then 'Si' else 'No' end as Jub_pen,
            case when fal.id_persona is not null then 'Si' else 'No' end as Fallecido
            from dat
            left join muebles aut on aut.ID_PERSONA = dat.id_persona
            left join inmuebles inm on inm.ID_PERSONA = dat.id_persona
            left join dep on dep.ID_PERSONA = dat.id_persona
            left join jub on jub.ID_PERSONA = dat.id_persona
            left join jur on jur.ID_PERSONA = dat.id_persona
            left join fal on fal.ID_PERSONA = dat.id_persona
            where 
            Inmuebles ='Si' 
            or Muebles ='Si' 
            or Servicio_Domestico ='Si' 
            or Dependiente ='Si'
            or Juridicas ='Si'
            or Jub_pen ='Si'
            or Fallecido='Si'
                """
)

#### ARMAMOS EL EXCEL DE SALIDA

In [ ]:
stats_prov.to_excel(writer, sheet_name="Stats globales")
eval.to_excel(writer, sheet_name="Efectores fuera del marco legal")
muebles.to_excel(writer, sheet_name="Detalle Muebles")
inmuebles.to_excel(writer, sheet_name="Detalle Inmuebles")
writer.save()